In [82]:
import torch
import time
import random
from nltk.tokenize import word_tokenize

import MyTrans as trans
from model import *
from utils import *
from dataloader import DataSet
from tqdm.notebook import tqdm as tqnb

In [83]:
params = {
    "model_dim": 64,
    "num_layers": 2,
    "num_heads": 8,
    "ff_dim": 128,
    "dropout": 0.1,
    "batch_size": 32,
    "learning_rate": 0.005,
    "num_epochs": 100,
    "seed": 2003,
    "model_path": "./models/",
    "early_stop": 10,
}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"use {device}")

seed = params["seed"]
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

use cuda


In [87]:
ds = DataSet.from_csv('./data/large_eng_fra.csv')
ds.set_split('test')
dl = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False, num_workers=1)

model = TransTrans(
    src_vocab_size=ds.get_src_vocab_size(),
    tgt_vocab_size=ds.get_tgt_vocab_size(),
    max_src_len=ds.get_max_src_len(),
    max_tgt_len=ds.get_max_tgt_len(),
    num_heads=params["num_heads"],
    num_encoders=params["num_layers"],
    num_decoders=params["num_layers"],
    dim_feedforward=params["ff_dim"],
    dim_model=params["model_dim"],
    device=device,
    padding_idx=ds.get_src_vocab().msk_idx,
    dropout=params["dropout"],
)
model.load_state_dict(torch.load('./models/model_2024-12-30-22-49-16_2003_e19_d10.pth'))
model.to(device)
model.eval()

/home/lishuang/anaconda3/envs/psc-cpi/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/tmp/ipykernel_1687889/1094213655.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` f

TransTrans(
  (src_emb): TransformerEmbedding(
    (tok_emb): Embedding(11971, 64, padding_idx=0)
    (pos_emb): PositionalEncoding()
    (drop_out): Dropout(p=0.1, inplace=False)
  )
  (tgt_emb): TransformerEmbedding(
    (tok_emb): Embedding(23457, 64, padding_idx=0)
    (pos_emb): PositionalEncoding()
    (drop_out): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
          )
          (linear1): Linear(in_features=64, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=128, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (

In [90]:
TT = torch.Tensor


def normalize_sizes(y_pred: TT, y_true: TT):
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    return y_pred, y_true


def compute_loss(y_pred: TT, y_true: TT, mask_idx):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return torch.nn.functional.cross_entropy(y_pred, y_true, ignore_index=mask_idx)


def compute_accuracy(y_pred: TT, y_true: TT, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)

    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()

    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100


loss_test, acc_test, batch_test = 0, 0, 0
test_bar = tqnb(total=len(dl), desc="test")

for data_dict in dl:
    x_src = data_dict["x_source"].to(device)
    x_tgt = data_dict["x_target"].to(device)
    y_tgt = data_dict["y_target"].to(device)
    len_x_src = data_dict["x_srclen"].to(device)
    len_x_tgt = data_dict["x_tgtlen"].to(device)

    with torch.no_grad():
        y_pred = model(x_src, x_tgt, len_x_src, len_x_tgt)

    loss = compute_loss(y_pred, y_tgt, mask_idx=ds.get_src_vocab().msk_idx)
    acc = compute_accuracy(y_pred, y_tgt, mask_index=ds.get_src_vocab().msk_idx)

    loss_test += loss.detach().cpu().item()
    acc_test += acc

    batch_test += 1

    test_bar.set_postfix(
        loss=loss_test / batch_test,
        acc=acc_test / batch_test,
    )
    test_bar.update()
    
test_bar.close()

print(f"Test Loss: {loss_test / batch_test:.4f}, Test Accuracy: {acc_test / batch_test:.2f}%")

test:   0%|          | 0/20377 [00:00<?, ?it/s]

Test Loss: 4.5496, Test Accuracy: 40.69%


In [88]:
def infer_target(model: TransTrans, src: str):
    src = src.lower()
    src_seq = word_tokenize(src)
    src_seq = " ".join(src_seq)
    vectorizor = ds.get_vectorizer()
    data_dict = vectorizor.vectorize(src_seq, "")

    x_src: TT = torch.tensor(data_dict["source_vector"]).unsqueeze(0).to(device)
    # x_tgt: TT = torch.tensor(data_dict["target_x_vector"]).unsqueeze(0).to(device)
    x_tgt: TT = torch.zeros_like(x_src).to(device)
    x_tgt[0, :] = vectorizor.tgt_vocab.msk_idx
    x_tgt[0, 0] = vectorizor.tgt_vocab.sos_idx
    x_src_len: TT = torch.tensor(data_dict["source_length"]).unsqueeze(0).to(device)
    x_tgt_len: TT = torch.tensor(1).unsqueeze(0).to(device)
    # x_tgt: TT = torch.tensor(data_dict["target_x_vector"]).unsqueeze(0).to(device)
    # x_tgt_len: TT = torch.tensor(data_dict["target_x_length"]).unsqueeze(0).to(device)

    while x_tgt_len[0] < vectorizor.max_tgt_len:
        y = model(x_src, x_tgt, x_src_len, x_tgt_len)
        y = torch.argmax(y, dim=-1)
        x_tgt[0, x_tgt_len[0]] = y[0, x_tgt_len[0] - 1]
        if x_tgt[0, x_tgt_len[0]] == vectorizor.tgt_vocab.eos_idx:
            break
        x_tgt_len[0] += 1

    # y = model(x_src, x_tgt, x_src_len, x_tgt_len)
    # y = torch.argmax(y, dim=-1)
    # x_tgt = y

    y_pred = [
        vectorizor.tgt_vocab.lookup_idx(token.item())
        for token in x_tgt[0, 1 : x_tgt_len[0]]
    ]
    return " ".join(y_pred)



In [89]:

src_list = [
    "I love you.",
    "He's out of town",
    "I'm glad you liked it.",
    "We're thinking of adding on another bedroom to the house.",
]

for sent in src_list:
    targ = infer_target(model, sent)
    print(f"{sent}")
    print(f"\t{targ}")

I love you.
	je t'aime .
He's out of town
	il est hors de la ville .
I'm glad you liked it.
	je suis heureux que vous l'ayez apprécié .
We're thinking of adding on another bedroom to the house.
	nous sommes en train de rêver .
